In [2]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import math

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/Ti-Yu/DFT/DFT.bit")
    ipDFT = ol.dft_0
    
    dftsize = 1024


    inBuffer0 = allocate(shape=(dftsize,), dtype=np.float32)
    inBuffer1 = allocate(shape=(dftsize,), dtype=np.float32)
    #outBuffer0 = allocate(shape=(dftsize,), dtype=np.float32)
    #outBuffer1 = allocate(shape=(dftsize,), dtype=np.float32)
    
    for i in range(dftsize):
        inBuffer0[i] = i
        inBuffer1[i] = 0
        


    
    timeKernelStart = time()
    ipDFT.write(0x10, inBuffer0.device_address)
    ipDFT.write(0x1c, inBuffer1.device_address)
    ipDFT.write(0x00, 0x01)
    while (ipDFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
      
        
        
    rmse_R = 0;
    rmse_I = 0;
    gold = open("/home/xilinx/Ti-Yu/DFT/out.gold.dat", "r+")
    for i in range(dftsize):
        line = gold.readline()
        id, re, img = line.split()
        re = float(re)
        img = float(img)
        rmse_R = rmse_R + (re - float(inBuffer0[i]))*(re - float(inBuffer0[i]))/dftsize
        rmse_I = rmse_I + (img - float(inBuffer1[i]))*(img - float(inBuffer1[i]))/dftsize
        #print(inBuffer0[i],inBuffer1[i])
    gold.close()
    rmse_R = math.sqrt(rmse_R)
    rmse_I = math.sqrt(rmse_I)
    
    print("----------------------------------------------\n");
    print("   RMSE(R)           RMSE(I)\n");
    print("%0.15f %0.15f\n", rmse_R, rmse_I);
    print("----------------------------------------------\n");

    if rmse_R > 0.1 or rmse_I > 0.1:
        print("*******************************************\n")
        print("FAIL: Output DOES NOT match the golden output\n")
        print("*******************************************\n")
    else:
        print("*******************************************\n")
        print("PASS: The output matches the golden output!\n")
        print("*******************************************\n")
    
    
    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Kernel execution time: 0.1264204978942871 s
----------------------------------------------

   RMSE(R)           RMSE(I)

%0.15f %0.15f
 0.0018532996396678832 0.001314949911643068
----------------------------------------------

*******************************************

PASS: The output matches the golden output!

*******************************************

Exit process
